In [ ]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

[I 2023-01-16 16:24:24,466.466 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-01-16 16:24:24,467.467 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


In [ ]:
%pip install pyEX

In [5]:
import pyEX as p
import pandas as pd
import numpy as np
import pyfolio as pf
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import awswrangler as wr
isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    token ='pk_2e94555e43da4135a6032433c6b18fa5'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

ModuleNotFoundError: No module named 'pyEX'

In [ ]:
fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
bitcoin_etf_symbols = ['BITO']
commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
bench_symbol=['SPY','QQQ','IVV']
all_symbols =fx_etf_symbols+bitcoin_etf_symbols+commod_etf_symbols+fi_etf_symbols+equity_etf_symbols+bench_symbol+sectors
#all_symbols
columns_to_capture = ['close','open','symbol']
data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'

In [ ]:
def get_full_file_name(data_folder,one_sym):
    return(f"{data_folder}{one_sym}.parquet")
def combine_new_and_stored_data(new_df,file_name,columns_to_capture):
    try:
        old_pd = pd.read_parquet(file_name)
        final_df=pd.concat([old_pd,new_df[columns_to_capture]]).drop_duplicates().sort_index()
    except Exception as l_exc:
        final_df=new_df.drop_duplicates().sort_index()
    return(final_df)
if False:
    for one_sym in all_symbols:
        try:
            new_df = pyEX_cl.chartDF(symbol=one_sym, timeframe=timeframe)
            file_name = get_full_file_name(data_folder,one_sym)
            final_df = combine_new_and_stored_data(new_df,file_name,columns_to_capture)
            assert final_df['symbol'].values[0]==one_sym, 'STOP!!!'
            assert one_sym in file_name, 'STOP!!!'
            final_df[columns_to_capture].to_parquet(file_name,index=True)
        except Exception as l_exc:
            print(f"exc:{one_sym}, {l_exc}")
        #break

In [ ]:
print(new_df.index.min(),new_df.index.max(),final_df.index.min(),final_df.index.max() )